In [1]:
import arcpy
import pandas as pd
import numpy as np
import getpass
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Encrypting location of file geodatabase for security purposes
print('Enter file path where file geodatabase is located (entry is encrypted):')
arcpy.env.workspace = str(getpass.getpass())

age_fields = [f.name for f in arcpy.ListFields("X01_AGE_AND_SEX")]
age = pd.DataFrame(arcpy.da.TableToNumPyArray("X01_AGE_AND_SEX", age_fields))

pov_fields = [f.name for f in arcpy.ListFields("X17_POVERTY")]
poverty = pd.DataFrame(arcpy.da.TableToNumPyArray("X17_POVERTY", pov_fields))

income_fields = [f.name for f in arcpy.ListFields("X19_INCOME")]
income = pd.DataFrame(arcpy.da.TableToNumPyArray("X19_INCOME", income_fields))

house_fields = [f.name for f in arcpy.ListFields("X25_HOUSING_CHARACTERISTICS")]
housing = pd.DataFrame(arcpy.da.TableToNumPyArray("X25_HOUSING_CHARACTERISTICS", house_fields))

insure_fields = [f.name for f in arcpy.ListFields("X27_HEALTH_INSURANCE")]
insurance = pd.DataFrame(arcpy.da.TableToNumPyArray("X27_HEALTH_INSURANCE", insure_fields))

Enter file path where file geodatabase is located (entry is encrypted):
········


In [3]:
col = ['B01001e' + str(n) for n in range (7,19+1)]
age["men18to64"] = age[col].sum(axis='columns')
col = ['B01001e' + str(n) for n in range (31,43+1)]
age["women18to64"] = age[col].sum(axis='columns')
age["total18to64"] = age["men18to64"]+age["women18to64"]

col = ['B01001e' + str(n) for n in range (7,12+1)]
age["men18to34"] = age[col].sum(axis='columns')
col = ['B01001e' + str(n) for n in range (31,36+1)]
age["women18to34"] = age[col].sum(axis='columns')
age["total18to34"] = age["men18to34"]+age["women18to34"]

col = ['B01001e' + str(n) for n in range (13,15+1)]
age["men35to49"] = age[col].sum(axis='columns')
col = ['B01001e' + str(n) for n in range (37,39+1)]
age["women35to49"] = age[col].sum(axis='columns')
age["total35to49"] = age["men35to49"]+age["women35to49"]

col = ['B01001e' + str(n) for n in range (16,19+1)]
age["men50to64"] = age[col].sum(axis='columns')
col = ['B01001e' + str(n) for n in range (40,43+1)]
age["women50to64"] = age[col].sum(axis='columns')
age["total50to64"] = age["men50to64"]+age["women50to64"]

col = ['B01001Be' + str(n) for n in range (7,13+1)]
age["menBlack"] = age[col].sum(axis='columns')
col = ['B01001Be' + str(n) for n in range (22,28+1)]
age["womenBlack"] = age[col].sum(axis='columns')
age["totalBlack"] = age["menBlack"]+age["womenBlack"]

col = ['B01001He' + str(n) for n in range (7,13+1)]
age["menWhite"] = age[col].sum(axis='columns')
col = ['B01001He' + str(n) for n in range (22,28+1)]
age["womenWhite"] = age[col].sum(axis='columns')
age["totalWhite"] = age["menWhite"]+age["womenWhite"]

col = ['B01001Ie' + str(n) for n in range (7,13+1)]
age["menLatino"] = age[col].sum(axis='columns')
col = ['B01001Ie' + str(n) for n in range (22,28+1)]
age["womenLatino"] = age[col].sum(axis='columns')
age["totalLatino"] = age["menLatino"]+age["womenLatino"]

age["other"] = (age["total18to64"] - (age["totalBlack"]+age["totalWhite"]+age["totalLatino"]))

age['entropy_denom'] = (age["totalLatino"]*1.00) + (age["totalWhite"]*1.00) + (age["totalBlack"]*1.00)

age['pWhite'] =(age["totalWhite"]/(age['entropy_denom']*1.00)).replace(np.inf, 0)

age['pBlack'] = (age["totalBlack"]/(age['entropy_denom']*1.00)).replace(np.inf, 0)

age['pLatino'] = (age["totalLatino"]/(age['entropy_denom']*1.00)).replace(np.inf, 0)

age['pBlkOrLat'] = ((age["totalLatino"]+age["totalBlack"])/(age['entropy_denom']*1.00)).replace(np.inf, 0)

age['entropy'] = -1*((age['pWhite']*np.log(age['pWhite']))+((age['pBlack']+age['pLatino'])*np.log((age['pBlack']+age['pLatino']))))

age['pWhite']=age['pWhite'].fillna(0)
age['pBlack']=age['pBlack'].fillna(0)
age['pLatino']=age['pLatino'].fillna(0)
age['pBlkOrLat']=age['pBlkOrLat'].fillna(0)
age['entropy']=age['entropy'].fillna(0)

income['Gini'] = income['B19083e1']

poverty['belowPov'] = (poverty['C17002e2']+poverty['C17002e3'])/poverty['C17002e1']
poverty['Pov1to149'] = (poverty['C17002e4']+poverty['C17002e5'])/poverty['C17002e1']
poverty['Pov150to199'] = (poverty['C17002e6']+poverty['C17002e7'])/poverty['C17002e1']
poverty['Pov2Plus'] = (poverty['C17002e8'])/poverty['C17002e1']

housing['noCar'] = (housing['B25044e3']+housing['B25044e10'])/housing['B25044e1']

insurance['private'] = (insurance['B27002e10'] + insurance['B27002e13'] +  insurance['B27002e16'] + insurance['B27002e19'] + insurance['B27002e22'] + insurance['B27002e38'] + insurance['B27002e41'] +  insurance['B27002e44'] + insurance['B27002e47'] + insurance['B27002e50'])/insurance['B27002e1']

s1 = pd.merge(age[['GEOID','men18to64','women18to64','total18to64',
                  'men18to34','women18to34','total18to34',
                  'men35to49','women35to49','total35to49',
                  'men50to64','women50to64','total50to64',
                  'menBlack','womenBlack','totalBlack',
                  'menWhite','womenWhite','totalWhite',
                  'menLatino','womenLatino','totalLatino','other',
                  'pWhite','pBlack','pLatino','pBlkOrLat','entropy_denom','entropy']],income[['GEOID','Gini']],
             how='left', on=['GEOID'])

s2 = pd.merge(s1, poverty[['GEOID','belowPov','Pov1to149',
                          'Pov150to199','Pov2Plus']],
             how='left', on=['GEOID'])

s3 = pd.merge(s2, housing[['GEOID','noCar']],
             how='left', on=['GEOID'])

censusResults = pd.merge(s3, insurance[['GEOID','private']],
             how='left', on=['GEOID'])

censusResults['tract'] = censusResults.GEOID.str[7:]
censusResults.head()

,GEOID,men18to64,women18to64,total18to64,men18to34,women18to34,total18to34,men35to49,women35to49,total35to49,...,entropy_denom,entropy,Gini,belowPov,Pov1to149,Pov150to199,Pov2Plus,noCar,private,tract
0,14000US01001020100,617.0,654.0,1271.0,226.0,180.0,406.0,161.0,187.0,348.0,...,1170.0,0.408362,0.4741,0.113365,0.086323,0.036921,0.763391,0.007843,0.538222,01001020100
1,14000US01001020200,749.0,578.0,1327.0,288.0,170.0,458.0,231.0,239.0,470.0,...,1292.0,0.681358,0.4121,0.178768,0.099560,0.162816,0.558856,0.123783,0.433828,01001020200
2,14000US01001020300,957.0,1098.0,2055.0,391.0,548.0,939.0,292.0,263.0,555.0,...,1993.0,0.622733,0.3943,0.150460,0.086881,0.138377,0.624281,0.076389,0.484010,01001020300
3,14000US01001020400,1029.0,1116.0,2145.0,354.0,339.0,693.0,368.0,289.0,657.0,...,2136.0,0.435171,0.3841,0.028452,0.095275,0.116941,0.759332,0.012813,0.479557,01001020400
4,14000US01001020500,2969.0,3021.0,5990.0,1039.0,1088.0,2127.0,1050.0,1126.0,2176.0,...,5675.0,0.503803,0.4928,0.151509,0.052117,0.027294,0.769080,0.055103,0.539242,01001020500
